# Lab 5: Land Surface Temperature using CORINE-based Emissivity

## 🎯 Objectives
In this exercise, you will:
- Select a cloud-free Landsat 8 images from 2013 and 2023 (or different if you're sure that you'll spot a difference in land cover)
- Calculate Brightness Temperature (TB) from Band 10.
- Load CORINE Land Cover data and assign emissivity values to each land cover class.
- Use the Planck-based formula to calculate Land Surface Temperature (LST).
- Visualize and interpret the results.

## Step 1: Define Area of Interest (AOI)
- Use coordinates around Reduta street in Kraków.
- Create a polygon or rectangle using `ee.Geometry.Polygon`.

In [80]:
import ee
ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize()

# Define the AOI using ee.Geometry.Rectangle
aoi = ee.Geometry.Rectangle([19.88, 50.010, 19.92, 50.030])  # Replace with actual coordinates

# Print the AOI to verify
print('AOI:', aoi.getInfo())

AOI: {'type': 'Polygon', 'coordinates': [[[19.88, 50.01], [19.92, 50.01], [19.92, 50.03], [19.88, 50.03], [19.88, 50.01]]]}


## Step 2: Load Landsat 8 imagery for the dates you've picked
- Filter for low cloud cover (< 20%)
- Select Band 10 and convert to TB using: `TB = ST_B10 * 0.00341802 + 149.0`

In [81]:
# Load Landsat 8 image collection
landsat_8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

# Filter images for 2013 and 2023 with low cloud cover (< 20%) and within the AOI
landsat_2013 = landsat_8.filterDate('2013-01-01', '2013-12-31') \
                        .filterBounds(aoi) \
                        .filter(ee.Filter.lt('CLOUD_COVER', 20))

landsat_2023 = landsat_8.filterDate('2023-01-01', '2023-12-31') \
                        .filterBounds(aoi) \
                        .filter(ee.Filter.lt('CLOUD_COVER', 20))

# Select Band 10 and calculate Brightness Temperature (TB)
def calculate_tb(image):
    tb = image.select('ST_B10').multiply(0.00341802).add(149.0)
    return image.addBands(tb.rename('TB'))

landsat_2013_tb = landsat_2013.map(calculate_tb)
landsat_2023_tb = landsat_2023.map(calculate_tb)

print(landsat_2013_tb.getInfo())
print(landsat_2023_tb.getInfo())

{'type': 'ImageCollection', 'bands': [], 'version': 1744929386365740.0, 'id': 'LANDSAT/LC08/C02/T1_L2', 'properties': {'type_name': 'ImageCollection', 'keywords': ['cfmask', 'cloud', 'fmask', 'global', 'l8sr', 'landsat', 'lasrc', 'lst', 'reflectance', 'sr', 'usgs'], 'visualization_1_bands': 'SR_B5,SR_B4,SR_B3', 'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png', 'visualization_1_max': '30000.0', 'description': '<p>This dataset contains atmospherically corrected\nsurface reflectance and land surface temperature derived from the data\nproduced by the Landsat 8 OLI/TIRS sensors.\nThese images contain 5 visible and near-infrared (VNIR) bands and\n2 short-wave infrared (SWIR) bands processed to orthorectified surface\nreflectance, and one thermal infrared (TIR) band processed to orthorectified\nsurface temperature. They also contain intermediate bands used in\ncalculation of the ST products, as well as QA bands.</p><p>Landsat 8 SR products are created with the Land Surface

## Step 3: Load CORINE Land Cover data
- Use dataset `COPERNICUS/CORINE/V20/100m/2018`
- Clip it to your AOI

In [82]:
# Load CORINE Land Cover dataset
corine = ee.Image('COPERNICUS/CORINE/V20/100m/2018')

# Clip the CORINE dataset to the AOI
corine_clipped = corine.clip(aoi)

# Display the clipped CORINE dataset
print(corine_clipped)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.clip",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    19.88,
                    50.03
                  ],
                  [
                    19.88,
                    50.01
                  ],
                  [
                    19.92,
                    50.01
                  ],
                  [
                    19.92,
                    50.03
                  ]
                ]
              ]
            },
            "evenOdd": {
              "constantValue": true
            }
          }
        }
      },
      "input": {
        "functionInvocationValue": {
          "functionName": "Image.load",
          "arguments": {
            "id": {
              "c

## Step 4: Assign emissivity to CORINE classes
- Use a dictionary for classes
- Use `remap()` and optionally a default value

In [83]:
# Create emissivity image
emissivity_dict = {
    111: 0.92,  # Continuous urban fabric
    112: 0.92,  # Discontinuous urban fabric
    121: 0.91,  # Industrial or commercial units
    211: 0.96,  # Non-irrigated arable land
    311: 0.98,  # Forests
    412: 0.97,  # Peat bogs
    324: 0.96,  # Transitional woodland-shrub
    231: 0.97   # Pastures
}


# Assign emissivity values to CORINE classes using the dictionary
emissivity_image = corine_clipped.remap(list(emissivity_dict.keys()),list(emissivity_dict.values()))

# Display the emissivity image
#print(emissivity_image)


## Step 5: Calculate LST using the formula:
$$
LST = \frac{T_B}{1 + \left( \frac{\lambda \cdot T_B}{c_2} \right) \cdot \ln(\varepsilon)}
$$
- λ = 10.8 µm
- c₂ = 14388 µm·K

In [84]:
# Create emissivity image
emissivity_dict = {
    111: 0.92,  # Continuous urban fabric
    112: 0.92,  # Discontinuous urban fabric
    121: 0.91,  # Industrial or commercial units
    211: 0.96,  # Non-irrigated arable land
    311: 0.98,  # Forests
    412: 0.97,  # Peat bogs
    324: 0.96,  # Transitional woodland-shrub
    231: 0.97   # Pastures
}

# Assign emissivity values to CORINE classes using the dictionary
emissivity_image = corine_clipped.remap(
    list(emissivity_dict.keys()), 
    list(emissivity_dict.values()), 
    0.95  # Default emissivity value
)

# Display the emissivity image
print(emissivity_image)
# Constants
lambda_wavelength = 10.8e-6  # Wavelength in meters (10.8 µm)
c2 = 0.014388  # c₂ in m·K (14388 µm·K)

# Ensure emissivity is an ee.Image before using it in calculations

# Update the calculate_lst function to use the corrected emissivity
def calculate_lst(image):
    tb = image.select('TB')  # Brightness Temperature
    lst = tb.divide(
        ee.Image(1).add(
            tb.multiply(lambda_wavelength).divide(c2).multiply(emissivity.log())
        )
    )
    return image.addBands(lst.rename('LST'))

# Recalculate LST for 2013 and 2023
landsat_2013_lst = landsat_2013_tb.map(calculate_lst)
landsat_2023_lst = landsat_2023_tb.map(calculate_lst)

# Display the LST image collections
print(landsat_2013_lst)
print(landsat_2023_lst)
# Ensure emissivity is defined as an ee.Image
emissivity = ee.Image(emissivity_image)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.remap",
    "arguments": {
      "defaultValue": {
        "constantValue": 0.95
      },
      "from": {
        "constantValue": [
          111,
          112,
          121,
          211,
          311,
          412,
          324,
          231
        ]
      },
      "image": {
        "functionInvocationValue": {
          "functionName": "Image.clip",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "GeometryConstructors.Polygon",
                "arguments": {
                  "coordinates": {
                    "constantValue": [
                      [
                        [
                          19.88,
                          50.03
                        ],
                        [
                          19.88,
                          50.01
                        ],
                        [
                 

## Step 6: Visualize the LST result
- Use palette: `['blue', 'yellow', 'red']`
- Suggested range: `min=290`, `max=325`

In [85]:
import geemap


# Initialize the map
Map = geemap.Map()


# Visualize LST for 2013
Map.addLayer(
    landsat_2013_lst.mean().select('LST'),
    {
        'min': 290,
        'max': 325,
        'palette': ['blue', 'yellow', 'red']
    },
    'LST 2013'
)

# Visualize LST for 2023
Map.addLayer(
    landsat_2023_lst.mean().select('LST'),
    {
        'min': 290,
        'max': 325,
        'palette': ['blue', 'yellow', 'red']
    },
    'LST 2023'
)

# Center the map on the AOI
Map.centerObject(aoi, 12)

# Display the map
Map


Map(center=[50.02000102479521, 19.899999999993007], controls=(WidgetControl(options=['position', 'transparent_…

## Step 7: (Optional) Analyze statistics by land cover class

In [ ]:
import geemap

# Import the required module for visualization

# Initialize the map
Map = geemap.Map()

# Visualize LST for 2013
Map.addLayer(
    landsat_2013_lst.mean().select('LST'),
    {
        'min': 290,
        'max': 325,
        'palette': ['blue', 'yellow', 'red']
    },
    'LST 2013'
)

# Visualize LST for 2023
Map.addLayer(
    landsat_2023_lst.mean().select('LST'),
    {
        'min': 290,
        'max': 325,
        'palette': ['blue', 'yellow', 'red']
    },
    'LST 2023'
)

# Center the map on the AOI
Map.centerObject(aoi, 12)

EEException: GeometryConstructors.Polygon: LinearRing requires at least 3 points.

In [ ]:
# Optionally compute zonal statistics

## Step 8: (Optional - Easter Egg :)) Generate your own Land Cover Classification using TerraTorch and foundation models*

Based on the example/tutorial: https://aiforgood.itu.int/event/workshop-earth-observation-foundation-models-with-prithvi-eo-2-0-and-terratorch/

*to earn 5.0 grade that will make a great impact on your final grade